In [1]:
# Import Dependencies and modules
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
from spacy.lang.en import English
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from string import punctuation
from collections import Counter
from io import StringIO
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn

import nltk
import glob
import errno
import os
import json

from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim import models
from gensim.models import CoherenceModel, TfidfModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt


%matplotlib inline

# Load Data

In [2]:
json_data = pd.DataFrame(columns = ['author', 'text', 'title'])

In [3]:
# Define path to text files that contain the novels
path = '/Users/ayankarim/Documents/Thinkful/Bootcamp/Final Capstone Opinion Mining/opinion_mining/Notebook/Topic Modelling Working Set/files/*.json'
files = glob.glob(path)

all_jsons = []

for filenames in files:
    with open(filenames, 'r') as f:
        file = json.load(f)
        all_jsons.append(file)


In [4]:
for dicts in all_jsons:
    df = pd.DataFrame(dicts)
    json_data = json_data.append(df)

In [5]:
json_data.head()

,author,text,title
0,[\n\t\t\t\t\t\tJulian Chokkattu\t\t\t\t\t],"[<p>Google’s <a href=""https://store.google.com...",\n\t\tGoogle will announce hardware on October...
1,[\n\t\t\t\t\t\tChristian de Looper\t\t\t\t\t],"[<p>Google finally unveiled the new <a href=""h...",\n\t\tHere’s how to buy the new Google Pixel 3...
2,[\n\t\t\t\t\t\tSimon Hill\t\t\t\t\t],[<p>If you plan to buy one of Google’s <a href...,\n\t\tThe best Pixel 3 cases and covers\t
3,[\n\t\t\t\t\t\tSimon Hill\t\t\t\t\t],"[<p>As the developer of Android, Google turns ...",\n\t\tGoogle Pixel 3 vs. Pixel 2 vs. Pixel: Pi...
4,[\n\t\t\t\t\t\tSimon Hill\t\t\t\t\t],[<p>There are plenty of contenders in the <a h...,\n\t\tGoogle Pixel 3 vs. Samsung Galaxy S9: Wh...


# Clean Data

In [6]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!

    text = str(text).replace("\n", "")
    text = str(text).replace("\t", "")
    text = str(text).replace("\\n", "")
    text = str(text).replace("\\t", "")
    text = str(text).replace("\\", "")
    text = str(text).replace("xa0", " ")
    text = str(text).replace("\'", "")
    text = re.sub("<p>", "", str(text))
    text = re.sub("</p>", "", str(text))
    text = re.sub("</a>", "", str(text))
    text = re.sub('<[^>]+>', "", str(text)) 
    text = str(text).replace("\\u2019", "")
    text = str(text).replace("\\u2013", "")
    text = str(text).replace("\\u2018", "")
    text = str(text).replace("\\u00a0", "")
    text = str(text).replace("\\u00a3", "")
    text = str(text).replace("\u2014", "")
    text = str(text).replace("\u201d", "")
    text = str(text).replace("\u201c", "")
    return text


In [7]:
# Define function to clean text
def clean_text(df):
    # Convert lists to strings and remove brackets
    df['text'] = df['text'].astype(str)
    df['author'] = df['author'].astype(str)

    df['text'] = df['text'].map(lambda x: x.strip('[]'))
    df['author'] = df['author'].map(lambda x: x.strip('[]'))

    # Clean text
    df['text'] = df['text'].apply(lambda x: text_cleaner(x))
    df['title'] = df['title'].apply(lambda x: text_cleaner(x))
    df['author'] = df['author'].apply(lambda x: text_cleaner(x))

In [8]:
# Clean Text
clean_text(json_data)

In [9]:
# Reset index
json_data = json_data.reset_index()
json_data = json_data.drop(['index'], axis=1)

In [10]:
# Visualize dataframe
json_data.head()

,author,text,title
0,Julian Chokkattu,Google’s annual hardware launch event will tak...,"Google will announce hardware on October 9, ne..."
1,Christian de Looper,Google finally unveiled the new Google Pixel 3...,Here’s how to buy the new Google Pixel 3 and G...
2,Simon Hill,If you plan to buy one of Google’s Pixel 3 sma...,The best Pixel 3 cases and covers
3,Simon Hill,"As the developer of Android, Google turns out ...",Google Pixel 3 vs. Pixel 2 vs. Pixel: Picking ...
4,Simon Hill,There are plenty of contenders in the Android ...,Google Pixel 3 vs. Samsung Galaxy S9: Which sm...


# Pre-Process Data for NLP

In [11]:
# # Tokenize text
# df['text'] = df.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)
# df['title'] = df.apply(lambda row: nltk.word_tokenize(row['title']), axis=1)

# # Remove Stopwords, or keep it, might be important for aspect based semantics
# stop = stopwords.words('english')
# df['text'] = df['text'].apply(lambda x: [item for item in x if item not in stop])
# df['title'] = df['title'].apply(lambda x: [item for item in x if item not in stop])

# # Lowercase everything
# df['text'] = df['text'].astype(str)
# df['text'] = df['text'].apply(lambda x: x.lower())

# df['title'] = df['title'].astype(str)
# df['title'] = df['title'].apply(lambda x: x.lower())

# # remove all punctuations
# df['text'] = df['text'].apply(lambda x: ''.join(c for c in x if c not in punctuation))
# df['title'] = df['title'].apply(lambda x: ''.join(c for c in x if c not in punctuation))

In [12]:

spacy.load('en')
parser = English()

# Function to tokenize text
def tokenize(text):
    lda_tokens = []
    tokens  = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc = True))
        


In [13]:
data = list(json_data['text'])

data_words = list(sent_to_words(data))

In [14]:
print(data_words[:1])

[['google', 'annual', 'hardware', 'launch', 'event', 'will', 'take', 'place', 'on', 'october', 'in', 'new', 'york', 'city', 'the', 'company', 'sent', 'out', 'invites', 'to', 'media', 'including', 'digital', 'trends', 'confirming', 'the', 'date', 'which', 'had', 'leaked', 'in', 'august', 'the', 'date', 'and', 'venue', 'are', 'change', 'of', 'pace', 'considering', 'the', 'past', 'two', 'google', 'october', 'events', 'have', 'taken', 'place', 'in', 'san', 'francisco', 'on', 'october', 'the', 'company', 'is', 'widely', 'expected', 'to', 'launch', 'slew', 'of', 'hardware', 'products', 'ranging', 'from', 'smartphones', 'to', 'smart', 'home', 'devices', 'the', 'highlights', 'will', 'be', 'the', 'pixel', 'and', 'pixel', 'xl', 'successors', 'to', 'last', 'year', 'critically', 'acclaimed', 'pixel', 'and', 'pixel', 'xl', 'smartphones', 'there', 'have', 'been', 'an', 'alarmingly', 'high', 'number', 'of', 'leaks', 'for', 'the', 'pixel', 'series', 'and', 'if', 'true', 'we', 'know', 'quite', 'lot', '

In [15]:
# Create Bigrams and Trigrams

# Build the models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# fast way to get a sentece clubbed as a bigram/trigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [16]:
print(trigram_mod[bigram_mod[data_words[0]]])

['google', 'annual', 'hardware', 'launch', 'event', 'will', 'take', 'place', 'on', 'october', 'in', 'new', 'york', 'city', 'the', 'company', 'sent', 'out', 'invites', 'to', 'media', 'including', 'digital_trends', 'confirming', 'the', 'date', 'which', 'had', 'leaked', 'in', 'august', 'the', 'date', 'and', 'venue', 'are', 'change', 'of', 'pace', 'considering', 'the', 'past', 'two', 'google', 'october', 'events', 'have', 'taken', 'place', 'in', 'san', 'francisco', 'on', 'october', 'the', 'company', 'is', 'widely', 'expected', 'to', 'launch', 'slew', 'of', 'hardware', 'products', 'ranging', 'from', 'smartphones', 'to', 'smart', 'home', 'devices', 'the', 'highlights', 'will', 'be', 'the', 'pixel', 'and', 'pixel', 'xl', 'successors', 'to', 'last_year', 'critically', 'acclaimed', 'pixel', 'and', 'pixel', 'xl', 'smartphones', 'there', 'have', 'been', 'an', 'alarmingly', 'high', 'number', 'of', 'leaks', 'for', 'the', 'pixel', 'series', 'and', 'if', 'true', 'we', 'know', 'quite', 'lot', 'about',

In [17]:
# Remove Stopwords, make bigrams and lemmatize
stop_words = stopwords.words('english')
stop_words.extend(['pixel', 'iphone', 'samsung', 'apple', 'essential', 'xs', 'max', 
                  'huawei', 'galaxy', 'note', 'moto', 'oneplus', 'android', 'mate', 'pro', 'lg', 'sony', 'razer', 'phone', 'company', 
                  'smartphone', 'google', 'thinq', 'nokia', 'htc', 'xperia', 'xz'])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [18]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['annual', 'hardware', 'launch', 'event', 'take', 'place', 'october', 'new', 'york', 'city', 'send', 'invite', 'medium', 'include', 'digital_trend', 'confirm', 'date', 'leak', 'august', 'date', 'venue', 'change', 'pace', 'consider', 'october', 'event', 'take', 'place', 'san', 'francisco', 'october', 'widely', 'expect', 'launch', 'slay', 'hardware', 'product', 'range', 'smartphone', 'smart', 'home', 'device', 'highlight', 'successor', 'last_year', 'critically', 'acclaim', 'smartphone', 'alarmingly', 'high', 'number', 'leak', 'series', 'true', 'know', 'quite', 'lot', 'phone', 'may', 'due', 'carelessness', 'recently', 'someone', 'leave', 'lyft', 'separately', 'group', 'russia', 'claim', 'get', 'hand', 'shipment', 'smartphone', 'even', 'post', 'unbox', 'video', 'show', 'everything', 'get', 'box', 'expect', 'notch', 'design', 'cutout', 'top', 'screen', 'house', 'front_fac', 'camera', 'garner', 'criticism', 'notch', 'look', 'unusually', 'large', 'small', 'may', 'traditional', 'design', 'sli

In [19]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [20]:
len(corpus[2])

316

In [21]:
# gather tfidf scores
tfidf = models.TfidfModel(corpus, id2word = id2word)

# filter low value words
low_value = 0.025

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    new_bow = [b for b in bow if b[0] not in low_value_words]

    #reassign        
    corpus[i] = new_bow

In [22]:
len(corpus[2])

198

In [23]:
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (5, 1), (6, 3), (7, 1), (8, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 2), (33, 2), (34, 2), (35, 2), (36, 3), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (44, 3), (45, 2), (46, 1), (47, 2), (48, 1), (49, 1), (50, 1), (51, 1), (53, 1), (54, 1), (55, 2), (56, 1), (57, 1), (58, 1), (59, 2), (60, 1), (62, 1), (65, 2), (66, 3), (67, 3), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (75, 1), (78, 1), (81, 2), (82, 1), (83, 4), (85, 1), (87, 1), (88, 2), (89, 1), (90, 1), (91, 2), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 3), (98, 1), (99, 1), (100, 1), (101, 1), (102, 3), (103, 1), (104, 1), (105, 1), (106, 1), (107, 2), (108, 1), (109, 1), (110, 1), (112, 4), (113, 3), (114, 1), (115, 1), (116, 1), (117, 1), (118, 1), (119, 1), (120, 1), (122, 1), (123, 2), (124, 1), (125, 1), (126, 1), (127, 1),

In [24]:
# Build the lda topic model

lda = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                      id2word=id2word,
                                      num_topics=3, 
                                      random_state=100,
                                      update_every=1,
                                      chunksize=50,
                                      passes=25,
                                      alpha='auto',
                                      per_word_topics=True)

1. topic 0
    - camera, display, feature, megapixel, lens, screen, big, device
    
    reliability
    
2. topic 1
    - screen, app, case, tap, go, option, display, setting, button, notification, select
    
    functionality
    
3. topic 2
    - camera, photo, device, work, user, issue, leak, show, app, really, update
    
    design

In [25]:
# Print the key words in the 10 topics
pprint(lda.print_topics(num_words=15))
doc_lda = lda[corpus]

[(0,
  '0.018*"camera" + 0.013*"phone" + 0.010*"good" + 0.009*"display" + '
  '0.008*"feature" + 0.007*"megapixel" + 0.007*"offer" + 0.007*"find" + '
  '0.006*"lens" + 0.006*"winner" + 0.006*"screen" + 0.005*"get" + 0.005*"come" '
  '+ 0.005*"big" + 0.005*"device"'),
 (1,
  '0.015*"screen" + 0.015*"app" + 0.015*"case" + 0.012*"tap" + 0.007*"go" + '
  '0.007*"want" + 0.007*"set" + 0.006*"home" + 0.006*"option" + '
  '0.006*"display" + 0.006*"setting" + 0.005*"button" + 0.005*"turn" + '
  '0.005*"notification" + 0.005*"select"'),
 (2,
  '0.008*"camera" + 0.008*"photo" + 0.006*"device" + 0.005*"even" + '
  '0.004*"take" + 0.004*"say" + 0.004*"work" + 0.004*"could" + 0.004*"user" + '
  '0.003*"issue" + 0.003*"leak" + 0.003*"show" + 0.003*"app" + 0.003*"really" '
  '+ 0.003*"update"')]


In [26]:
# Evaluate how good the model is

# Compute perplexity
print('\nPerplexity: ', lda.log_perplexity(corpus)) # Lower the better

# Compute Coherence
coherenece_model_lda = CoherenceModel(model=lda, texts=data_lemmatized, dictionary=id2word, coherence='c_v' )
coherence_lda = coherenece_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.761362401661696

Coherence Score:  0.37494692931064205


In [27]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, id2word)
vis

/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.125796  0.118297       1        1  49.617039
2     -0.074071 -0.140633       2        1  35.698662
1      0.199868  0.022337       3        1  14.684300, topic_info=     Category        Freq          Term       Total  loglift  logprob
term                                                                 
86    Default  549.000000         phone  549.000000  30.0000  30.0000
145   Default  238.000000          case  238.000000  29.0000  29.0000
9     Default  346.000000           app  346.000000  28.0000  28.0000
102   Default  512.000000        screen  512.000000  27.0000  27.0000
658   Default  176.000000           tap  176.000000  26.0000  26.0000
309   Default  417.000000          good  417.000000  25.0000  25.0000
568   Default  298.000000     megapixel  298.000000  24.0000  24.0000
308   Default  162.000000            go  162.000000  23.0000  23.0000
469   Default  211.000000          want  211.000000  22.0000  22.0000
675   Default  264.000000        winner  264.000000  21.0000  21.0000
17    Default  977.000000        camera  977.000000  20.0000  20.0000
362   Default  151.000000        option  151.000000  19.0000  19.0000
158   Default  376.000000       feature  376.000000  18.0000  18.0000
193   Default  119.000000           set  119.000000  17.0000  17.0000
59    Default  167.000000          home  167.000000  16.0000  16.0000
84    Default  283.000000         offer  283.000000  15.0000  15.0000
25    Default  209.000000          come  209.000000  14.0000  14.0000
981   Default  102.000000       setting  102.000000  13.0000  13.0000
581   Default  270.000000         photo  270.000000  12.0000  12.0000
36    Default  485.000000       display  485.000000  11.0000  11.0000
286   Default  292.000000          find  292.000000  10.0000  10.0000
181   Default  190.000000         price  190.000000   9.0000   9.0000
326   Default  183.000000          inch  183.000000   8.0000   8.0000
127   Default  109.000000          turn  109.000000   7.0000   7.0000
265   Default  172.000000          dual  172.000000   6.0000   6.0000
949   Default   74.000000  notification   74.000000   5.0000   5.0000
73    Default  173.000000          look  173.000000   4.0000   4.0000
223   Default  193.000000          back  193.000000   3.0000   3.0000
1260  Default   62.000000        select   62.000000   2.0000   2.0000
426   Default  171.000000         still  171.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
909    Topic3   17.752037            gt   20.422913   1.7782  -6.5386
2948   Topic3   15.306606        scroll   17.395947   1.7904  -6.6868
388    Topic3   51.305897    protection   69.222801   1.6189  -5.4773
981    Topic3   72.138557       setting  102.829018   1.5639  -5.1365
193    Topic3   82.068230           set  119.464516   1.5429  -5.0075
989    Topic3   24.976223        slider   30.440645   1.7205  -6.1972
9      Topic3  182.856140           app  346.733551   1.2785  -4.2064
932    Topic3   25.743689          lock   33.519539   1.6545  -6.1669
308    Topic3   87.161751            go  162.641586   1.2946  -4.9473
387    Topic3   29.279488       protect   39.830688   1.6106  -6.0382
127    Topic3   61.739422          turn  109.375305   1.3465  -5.2922
102    Topic3  187.487595        screen  512.653076   0.9125  -4.1814
362    Topic3   75.902847        option  151.891388   1.2247  -5.0856
59     Topic3   79.700287          home  167.869949   1.1735  -5.0368
505    Topic3   43.604782        choose   72.927277   1.4041  -5.6399
3493   Topic3   22.060986         mouse   27.635639   1.6931  -6.3213
469    Topic3   83.151070          want  211.931091   0.9828  -4.9944
410    Topic3   25.177641      shortcut   34.880371   1.5924  -6.1891
480    Topic3   47.925262         allow   98.777756   1.1952  -5.5454
238    Topic3   62.410641        button  163.13

In [28]:
# Mallet LDA

mallet_path = '/Users/ayankarim/mallet-2.0.8/bin/mallet'

ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=3, id2word=id2word, iterations=1000, random_seed=100 )

1. topic 0
    - device, photo, year, issue, big, people, buy, price, user, mobile
    
    reliability
    
2. topic 1
    - camera, feature, lens, megapixel, display, update, design, play
    
    design
    
3. topic 2
    - screen, app, case, mode, tap, button, display, option, home, time, battery, setting, light
    
    functionality

In [29]:
# Show topics
pprint(ldamallet.show_topics(num_words=15, formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(0,
  [('device', 0.012748117727583847),
   ('photo', 0.010438056125941135),
   ('thing', 0.007486310746064339),
   ('year', 0.006930184804928132),
   ('good', 0.006160164271047228),
   ('issue', 0.00607460643394935),
   ('big', 0.006031827515400411),
   ('people', 0.005946269678302533),
   ('buy', 0.005775154004106776),
   ('feel', 0.005775154004106776),
   ('price', 0.0053901437371663245),
   ('user', 0.0053901437371663245),
   ('mobile', 0.005176249144421629),
   ('rear', 0.005047912388774812),
   ('handset', 0.004791238877481177)]),
 (1,
  [('camera', 0.036118409528814156),
   ('phone', 0.02161499386932913),
   ('feature', 0.012401471361008934),
   ('lens', 0.011770888071466106),
   ('good', 0.010965142757050272),
   ('megapixel', 0.010895077947101069),
   ('display', 0.010825013137151866),
   ('offer', 0.010509721492380452),
   ('find', 0.010404624277456647),
   ('winner', 0.009143457698370994),
   ('update', 0.008302679978980557),
   ('back', 0.007286740234717114),
   ('inch', 0

In [51]:
from gensim import corpora, models, similarities
from itertools import chain

# Prints the topics.
for top in lda.print_topics():
    print (top)

# Assigns the topics to the documents in corpus
lda_corpus = lda[corpus]

# Find the threshold, let's set the threshold to be 1/#clusters,
# To prove that the threshold is sane, we average the sum of all probabilities:
scores = list(chain(*[[score for topic_id, score in topic] \
                      for topic in [doc for doc in lda_corpus]]))

threshold = sum(scores)/len(scores)
print (threshold)


cluster1 = [j for i,j in zip(lda_corpus,documents) if i[0][1] > threshold]
cluster2 = [j for i,j in zip(lda_corpus,documents) if i[1][1] > threshold]
cluster3 = [j for i,j in zip(lda_corpus,documents) if i[2][1] > threshold]
cluster4 = [j for i,j in zip(lda_corpus,documents) if i[3][1] > threshold]

(0, '0.018*"camera" + 0.013*"phone" + 0.010*"good" + 0.009*"display" + 0.008*"feature" + 0.007*"megapixel" + 0.007*"offer" + 0.007*"find" + 0.006*"lens" + 0.006*"winner"')
(1, '0.015*"screen" + 0.015*"app" + 0.015*"case" + 0.012*"tap" + 0.007*"go" + 0.007*"want" + 0.007*"set" + 0.006*"home" + 0.006*"option" + 0.006*"display"')
(2, '0.008*"camera" + 0.008*"photo" + 0.006*"device" + 0.005*"even" + 0.004*"take" + 0.004*"say" + 0.004*"work" + 0.004*"could" + 0.004*"user" + 0.003*"issue"')


ValueError: too many values to unpack (expected 2)

In [52]:
print([i for i in lda_corpus])

[([(0, 0.18610248), (2, 0.8135868)], [(0, [2, 0]), (1, [2, 0]), (2, [2, 0]), (3, [2, 0]), (5, [2, 0]), (6, [2, 0]), (7, [2, 0]), (8, [2]), (10, [2, 0]), (11, [2, 0]), (12, [0, 2]), (13, [2, 0]), (14, [2, 0]), (15, [2]), (16, [2, 0]), (18, [2, 0]), (19, [2, 0]), (20, [2]), (21, [2]), (22, [2, 0]), (23, [2, 0]), (24, [2]), (26, [2, 0]), (27, [2, 0]), (28, [2, 0]), (29, [0, 2]), (30, [0, 2]), (31, [2, 0]), (32, [2, 0]), (33, [0, 2]), (34, [2, 0]), (35, [2]), (36, [0, 2]), (37, [2, 0]), (38, [2, 0]), (39, [2, 0]), (40, [2, 0]), (41, [2]), (42, [2, 0]), (44, [2, 0]), (45, [2, 0]), (46, [2, 0]), (47, [0]), (48, [2, 0]), (49, [2]), (50, [0]), (51, [2]), (53, [2, 0]), (54, [0, 2]), (55, [2, 0]), (56, [2, 0]), (57, [2, 0]), (58, [2, 0]), (59, [2, 0]), (60, [0, 2]), (62, [2]), (65, [2, 0]), (66, [2, 0]), (67, [2]), (68, [2, 0]), (69, [2, 0]), (70, [2, 0]), (71, [2, 0]), (72, [2, 0]), (75, [2]), (78, [2, 0]), (81, [2, 0]), (82, [2, 0]), (83, [2, 0]), (85, [2, 0]), (87, [2]), (88, [2, 0]), (89, [2

In [ ]:
    # IPhone X
    # S9
    # Pixel 3
    # Huawei Mate 20 Pro 
    # OnePlus 6T
    # Huawei P2 Pro
    # LG V4 Thinq
    # Sony Xperia XZ3
    # Essential Phone
    # Razer Phone 2
    # HTC U12+
    # Moto G6 plus

In [ ]:
## Assign the topics to the texts

## Pickle dump the topic models
## Call the models the in the opinion mining notebook and use them to classify the texts into topics
## Use Multilabel Naive Bayes to get an output of aspect terms in a dataframe grouped into topic columns, ordered by score

### HOW TO EVALUATE YOUR MODEL!!??

## Clean the Notebook, write up a report
## Create Presentation